# Spatial Analysis: San Jose (DBSCAN)

**Project:** A Tale of Two Cities - Comparative Public Safety Analysis

**Purpose:** This notebook handles the spatial analysis for San Jose. It uses DBSCAN clustering to identify public safety hotspots from the processed data.

**Input:** `data/processed/sj_calls_cleaned.csv`

**Analysis:**
1.  Load the cleaned data.
2.  Sample the data for efficient clustering.
3.  Determine optimal DBSCAN parameters (`eps`).
4.  Run DBSCAN to find clusters.
5.  Visualize the resulting clusters on a map.

In [3]:
print("Hello, World!")

Hello, World!


## 1. Import Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# Plot settings
plt.rcParams["figure.figsize"] = (14, 10)
plt.rcParams["axes.grid"] = True
sns.set_style("whitegrid")

print("Libraries imported successfully")

Matplotlib is building the font cache; this may take a moment.


Libraries imported successfully


## 2. Load Data

In [5]:
# Load cleaned SJ calls data
# Note: This file was created by the '01_data_preparation_sj.ipynb' notebook
df = pd.read_csv(
    '../data/processed/sj_calls_cleaned.csv',
    index_col='Incident DateTime',
    parse_dates=True
)

print(f"Dataset loaded: {len(df):,} rows")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"\nColumns: {df.columns.tolist()}")

# Display basic statistics
# We must check if Latitude and Longitude columns exist
if 'Latitude' in df.columns and 'Longitude' in df.columns:
    print("\nSpatial Data Summary:")
    print(df[['Latitude', 'Longitude']].describe())
else:
    print("\n\nCRITICAL ERROR: 'Latitude' or 'Longitude' columns not found in the data.")
    print("Please run the updated '01_data_preparation_sj.ipynb' with the geocoding step first.")

Dataset loaded: 1,168,824 rows
Date range: 2018-01-01 00:00:02 to 2025-11-08 23:55:37

Columns: ['Incident_High_Level_Category', 'Resolution', 'Neighborhood', 'Police_District', 'Hour', 'Day', 'Month', 'Year', 'Day_of_Week', 'Day_of_Week_Name', 'Month_Name', 'Quarter', 'Is_Weekend']


CRITICAL ERROR: 'Latitude' or 'Longitude' columns not found in the data.
Please run the updated '01_data_preparation_sj.ipynb' with the geocoding step first.


## 3. Exploratory Spatial Analysis

In [ ]:
# This cell will fail if Lat/Lon are missing
if 'Latitude' in df.columns and 'Longitude' in df.columns:
    print(f"Total incidents: {len(df):,}")
    print(f"Latitude range: [{df['Latitude'].min():.6f}, {df['Latitude'].max():.6f}]")
    print(f"Longitude range: [{df['Longitude'].min():.6f}, {df['Longitude'].max():.6f}]")

    # Create hexbin density plot (better for 1M+ points)
    plt.figure(figsize=(14, 10))
    plt.hexbin(df['Longitude'], df['Latitude'], gridsize=50, cmap='YlOrRd', mincnt=1)
    plt.colorbar(label='Incident Count')
    plt.xlabel('Longitude', fontsize=12)
    plt.ylabel('Latitude', fontsize=12)
    plt.title('San Jose Incident Density Heatmap', fontsize=16, weight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("Cannot perform Exploratory Spatial Analysis: Missing coordinate data.")

## 4. Prepare Data for DBSCAN

In [ ]:
# This cell will fail if Lat/Lon are missing
if 'Latitude' in df.columns and 'Longitude' in df.columns:
    # Extract coordinates for clustering
    coords = df[['Latitude', 'Longitude']].values

    print(f"Total incidents for clustering: {len(coords):,}")
    print(f"Coordinates shape: {coords.shape}")

    # For computational efficiency, we'll sample 100,000 points
    MAX_SAMPLES = 100000

    print(f"\nDataset is large ({len(coords):,} points). Sampling {MAX_SAMPLES:,} points for analysis...")
    np.random.seed(42) # Use same seed as Tushar for reproducibility
    sample_indices = np.random.choice(len(coords), size=MAX_SAMPLES, replace=False)
    
    coords_sample = coords[sample_indices]
    df_sample = df.iloc[sample_indices].copy()
    
    print(f"Sample created: {len(coords_sample):,} points")
else:
    print("Cannot prepare data for DBSCAN: Missing coordinate data.")

## 5. Determine Optimal DBSCAN Parameters

In [ ]:
# This cell will fail if Lat/Lon are missing
if 'coords_sample' in locals():
    # Use k-nearest neighbors to determine optimal eps
    min_samples = 4 # Same as Tushar
    
    print(f"Computing k-nearest neighbors for {len(coords_sample):,} points...")
    neighbors = NearestNeighbors(n_neighbors=min_samples)
    neighbors_fit = neighbors.fit(coords_sample)
    distances, indices = neighbors_fit.kneighbors(coords_sample)

    # Sort distances
    distances = np.sort(distances[:, min_samples-1], axis=0)

    # Plot k-distance graph
    plt.figure(figsize=(12, 6))
    plt.plot(distances)
    plt.xlabel('Points sorted by distance', fontsize=12)
    plt.ylabel(f'{min_samples}-th Nearest Neighbor Distance', fontsize=12)
    plt.title('K-Distance Graph for Optimal Eps Selection', fontsize=16, weight='bold')
    plt.grid(True, alpha=0.3)
    
    # Add a zoomed-in plot, as the 'elbow' will be very low
    plt.figure(figsize=(12, 6))
    plt.plot(distances)
    plt.xlabel('Points sorted by distance', fontsize=12)
    plt.ylabel(f'{min_samples}-th Nearest Neighbor Distance', fontsize=12)
    plt.title('K-Distance Graph (Zoomed In)', fontsize=16, weight='bold')
    plt.ylim(0, 0.05) # Adjust this Y-limit to find your elbow
    plt.grid(True, alpha=0.3)
    plt.show()

    print("\nLook for the 'elbow' in the zoomed graph to determine optimal eps.")
else:
    print("Cannot determine DBSCAN parameters: Sample coordinate data not created.")

## 6. Apply DBSCAN Clustering

In [ ]:
# This cell will fail if Lat/Lon are missing
if 'coords_sample' in locals():
    # Set DBSCAN parameters (use the 'eps' value from your graph)
    eps = 0.01  # <<<--- ADJUST THIS based on your k-distance graph
    min_samples = 4

    print(f"Running DBSCAN with eps={eps} and min_samples={min_samples}...")
    print(f"Clustering {len(coords_sample):,} incidents...")

    # Apply DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
    clusters = dbscan.fit_predict(coords_sample)

    # Add cluster labels to dataframe
    df_sample['Cluster'] = clusters

    # Analyze results
    n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    n_noise = list(clusters).count(-1)
    n_clustered = len(clusters) - n_noise

    print("\n" + "="*60)
    print("DBSCAN CLUSTERING RESULTS")
    print("="*60)
    print(f"Number of clusters found: {n_clusters:,}")
    print(f"Noise points (outliers): {n_noise:,} ({100*n_noise/len(clusters):.2f}%)")
    print(f"Clustered points: {n_clustered:,} ({100*n_clustered/len(clusters):.2f}%)")
    
    if n_clusters > 0:
        print(f"Average cluster size: {n_clustered/n_clusters:.1f} incidents")
        # Display cluster size distribution
        cluster_sizes = df_sample[df_sample['Cluster'] != -1].groupby('Cluster').size()
        print(f"\nCluster size statistics:")
        print(f"  Min: {cluster_sizes.min():,} incidents")
        print(f"  Max: {cluster_sizes.max():,} incidents")
        print(f"  Median: {cluster_sizes.median():.0f} incidents")
        print(f"  Mean: {cluster_sizes.mean():.1f} incidents")
else:
    print("Cannot apply DBSCAN: Sample coordinate data not created.")

## 7. Visualize Clusters

In [ ]:
# This cell will fail if Lat/Lon are missing
if 'df_sample' in locals() and 'Cluster' in df_sample.columns:
    plt.figure(figsize=(16, 12))

    # Plot noise points in gray
    noise_mask = df_sample['Cluster'] == -1
    plt.scatter(
        df_sample.loc[noise_mask, 'Longitude'],
        df_sample.loc[noise_mask, 'Latitude'],
        c='lightgray',
        alpha=0.3,
        s=5,
        label='Noise/Outliers'
    )

    # Plot clustered points with colors
    clustered_mask = df_sample['Cluster'] != -1
    if clustered_mask.sum() > 0:
        scatter = plt.scatter(
            df_sample.loc[clustered_mask, 'Longitude'],
            df_sample.loc[clustered_mask, 'Latitude'],
            c=df_sample.loc[clustered_mask, 'Cluster'],
            cmap='tab20', # A colormap with many distinct colors
            alpha=0.6,
            s=10,
            edgecolors='none'
        )
        plt.colorbar(scatter, label='Cluster ID')

    plt.xlabel('Longitude', fontsize=12)
    plt.ylabel('Latitude', fontsize=12)
    plt.title(f'San Jose DBSCAN Clustering Results (eps={eps}, min_samples={min_samples})', 
              fontsize=16, weight='bold')
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    # Save figure
    output_dir = Path('../reports/figures')
    output_dir.mkdir(parents=True, exist_ok=True)
    plt.savefig(output_dir / '07_sj_dbscan_clusters_overview.png', dpi=300, bbox_inches='tight')

    plt.show()
else:
    print("Cannot visualize clusters: Clustered dataframe not available.")

## 8. Analyze Top Hotspot Clusters

In [ ]:
# This cell will fail if Lat/Lon are missing
if 'df_sample' in locals() and n_clusters > 0:
    cluster_info = df_sample[df_sample['Cluster'] != -1].groupby('Cluster').agg({
        'Latitude': ['mean', 'count'],
        'Longitude': 'mean',
        'Neighborhood': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown'
    }).round(6)
    
    cluster_info.columns = ['Center_Lat', 'Size', 'Center_Lon', 'Neighborhood']
    cluster_info = cluster_info.sort_values('Size', ascending=False)
    
    print("\n" + "="*80)
    print("TOP 20 SAN JOSE INCIDENT HOTSPOTS (Largest Clusters)")
    print("="*80)
    print(cluster_info.head(20).to_string())
    
else:
    print("No clusters found to analyze.")

## 9. Summary and Key Findings

In [ ]:
# This cell will fail if Lat/Lon are missing
if 'df_sample' in locals() and n_clusters > 0:
    print("="*80)
    print("SPATIAL ANALYSIS SUMMARY - SAN JOSE INCIDENTS")
    print("="*80)
    print(f"\n📊 Dataset Information:")
    print(f"   • Total incidents in source file: {len(df):,}")
    print(f"   • Incidents sampled for analysis: {len(df_sample):,}")
    print(f"   • Date range: {df_sample.index.min().date()} to {df_sample.index.max().date()}")

    print(f"\n🎯 DBSCAN Parameters:")
    print(f"   • Epsilon (eps): {eps}")
    print(f"   • Minimum samples: {min_samples}")

    print(f"\n🗺️ Clustering Results:")
    print(f"   • Number of clusters identified: {n_clusters:,}")
    print(f"   • Incidents in clusters: {n_clustered:,} ({100*n_clustered/len(clusters):.1f}%)")
    print(f"   • Noise/outlier points: {n_noise:,} ({100*n_noise/len(clusters):.1f}%)")

    if n_clusters > 0:
        print(f"\n📈 Cluster Statistics:")
        print(f"   • Average cluster size: {cluster_sizes.mean():.1f} incidents")
        print(f"   • Median cluster size: {cluster_sizes.median():.0f} incidents")
        print(f"   • Largest cluster: {cluster_sizes.max():,} incidents")
        print(f"   • Smallest cluster: {cluster_sizes.min():,} incidents")
        
        print(f"\n🔥 Top 5 Hotspots (by Neighborhood):")
        for i, (cluster_id, row) in enumerate(cluster_info.head(5).iterrows(), 1):
            print(f"   {i}. Cluster {cluster_id}: {row['Size']:,} incidents - {row['Neighborhood']}")

    print("\n" + "="*80)
else:
    print("Cannot generate summary: Clustering was not performed.")